In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.number import inverse
from Crypto.Util.Padding import pad, unpad
from collections import namedtuple
from random import randint
import hashlib
import os

In [2]:
# Create a simple Point class to represent the affine points.
Point = namedtuple("Point", "x y")

# The point at infinity (origin for the group law).
O = 'Origin'

FLAG = b'crypto{??????????????????????????????}'

In [4]:
def check_point(P: tuple):
    if P == O:
        return True
    else:
        return (P.y**2 - (P.x**3 + a*P.x + b)) % p == 0 and 0 <= P.x < p and 0 <= P.y < p


def point_inverse(P: tuple):
    if P == O:
        return P
    return Point(P.x, -P.y % p)


def point_addition(P: tuple, Q: tuple):
    # based of algo. in ICM
    if P == O:
        return Q
    elif Q == O:
        return P
    elif Q == point_inverse(P):
        return O
    else:
        if P == Q:
            lam = (3*P.x**2 + a)*inverse(2*P.y, p)
            lam %= p
        else:
            lam = (Q.y - P.y) * inverse((Q.x - P.x), p)
            lam %= p
    Rx = (lam**2 - P.x - Q.x) % p
    Ry = (lam*(P.x - Rx) - P.y) % p
    R = Point(Rx, Ry)
    assert check_point(R)
    return R


def double_and_add(P: tuple, n: int):
    # based of algo. in ICM
    Q = P
    R = O
    while n > 0:
        if n % 2 == 1:
            R = point_addition(R, Q)
        Q = point_addition(Q, Q)
        n = n // 2
    assert check_point(R)
    return R

In [5]:
def gen_shared_secret(Q: tuple, n: int):
    # Bob's Public key, my secret int
    S = double_and_add(Q, n)
    return S.x


def encrypt_flag(shared_secret: int):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    # Encrypt flag
    iv = os.urandom(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(FLAG, 16))
    # Prepare data to send
    data = {}
    data['iv'] = iv.hex()
    data['encrypted_flag'] = ciphertext.hex()
    return data

In [16]:
# Define the curve
p = 310717010502520989590157367261876774703
a = 2
b = 3

# Generator
g_x = 179210853392303317793440285562762725654
g_y = 105268671499942631758568591033409611165
G = E(g_x, g_y)

In [14]:
E = EllipticCurve(GF(p), [a,b])


In [9]:
a_x = 280810182131414898730378982766101210916
a_y = 291506490768054478159835604632710368904
A = E(a_x, a_y)

In [21]:
n = discrete_log(A, G, operation = '+', algorithm = 'bsgs')

In [17]:
b_x = 272640099140026426377756188075937988094
b_y = 51062462309521034358726608268084433317
B = E(b_x, b_y)

In [32]:
shared_point = n*B
shared_secret = shared_point.x()

In [51]:
ciphertext = bytes.fromhex("8220b7c47b36777a737f5ef9caa2814cf20c1c1ef496ec21a9b4833da24a008d0870d3ac3a6ad80065c138a2ed6136af")
iv = bytes.fromhex("07e2628b590095a5e332d397b8a59aa7")

In [52]:
def decrypt(ciphertext: bytes, shared_secret: int, iv: bytes) -> bytes:
    # Step 1: Derive AES key
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))  # must be bytes
    key = sha1.digest()[:16]  # must be 16 bytes

    # Step 2: Decrypt using AES CBC
    cipher = AES.new(key, AES.MODE_CBC, iv)  # iv must be bytes
    padded_plaintext = cipher.decrypt(ciphertext)
    plaintext = unpad(padded_plaintext, 16)
    return plaintext

In [54]:
plaintext = decrypt(ciphertext, shared_secret, iv)
print(plaintext)

b'crypto{n07_4ll_curv3s_4r3_s4f3_curv3s}'
